# Build Merged Table
To begin our analysis, we'll build a merged table of reads, that we'll use for most of the following analyses. This will collect all data from the data directory, and build a table with all reads, annotated with metadata. 


To ensure the following cells work properly, please ensure all relevant data has been downloaded from Zenodo (10.5281/zenodo.11038446). Please reference the ReadMe file to sort data into proper folders in the GitHub directory. 
__________

In [1]:
import pandas as pd
from utils import *

## Taxonomy Ranks
Define the taxonomy ranks to collapse on for all reads

In [2]:
collapse_on=["kingdom", "phylum", "class", "order", "family", "genus"]

## Nasopharyx Samples
We'll start with the nasopharynx samples

In [ ]:
# initalize dataframe 
res = pd.DataFrame()

# collect list of NP studies
manifest_NP = pd.read_csv('../data/NP/NP_manifest.csv',index_col= 0, header = None)

for file_name in manifest_NP.index:
    
    # pull out feature tables with total reads
    ab = qiime_to_dataframe(feature_table="../data/NP/"+file_name+"_table.qza",
                        taxonomy="../data/NP/"+file_name+"_taxonomy.qza", 
                        collapse_on=collapse_on) 
    
    # merge with metadata
    meta = pd.read_csv('../data/NP/'+file_name+'_metadata.tsv', sep="\t")
    meta.rename(columns={meta.columns[0]: "sample_id"}, inplace=True)
    ab = pd.merge(ab, meta, on="sample_id")
    
    # identify URT site
    ab['URT'] = 'NP'
    
    # identify study name
    ab['study'] = file_name
    
    # concatenate with original dataframe
    res = pd.concat([res,ab])

## Oropharynx Samples
Next we'll add the oropharynx samples

In [ ]:
# collect list of OP studies 
manifest_OP = pd.read_csv('../data/OP/OP_manifest.csv',index_col= 0, header = None)

for file_name in manifest_OP.index:
    
    # pull out feature tables with total reads
    ab = qiime_to_dataframe(feature_table="../data/OP/"+file_name+"_table.qza",
                        taxonomy="../data/OP/"+file_name+"_taxonomy.qza", 
                        collapse_on=collapse_on) 
    
    # merge with metadata
    meta = pd.read_csv('../data/OP/'+file_name+'_metadata.tsv', sep="\t")
    meta.rename(columns={meta.columns[0]: "sample_id"}, inplace=True)
    ab = pd.merge(ab, meta, on="sample_id")
    
    # identify URT area
    ab['URT'] = 'OP'
    
    # identify study name
    ab['study'] = file_name
    
    # concatenate with original dataframe
    res = pd.concat([res,ab])

## Format Table
Now we'll format the table, calculate the centered-log ratio for each read, and add more metadata.  

In [ ]:
# drop identified reads or none bacterial reads
res = res.dropna(subset = ['genus'])
res = res[~(res.genus.str.contains('None'))&~(res.genus.str.contains('uncultured'))&
          ~(res.genus.str.contains('Chloroplast'))&~(res.family.str.contains('Mitochondria'))]

# centered log-ratio transformation and filtering
res = clr(filter_taxa(res, min_reads=2, min_prevalence=0.05)) 

# add metadata pertaining to studies 
studies = pd.read_csv('../data/studies.csv',header = None, index_col = 0)
res['disease'] = res['study'].map(studies[2].to_dict())
res['authors'] = res['study'].map(studies[1].to_dict())
res['study'] = res['authors']+', '+res['URT']

In [ ]:
res.to_csv('../data/merged_table.csv')